In [1]:
# cows missing teats
import os
import pandas as pd
import numpy as np 

In [27]:
dirpath = os.getcwd()
data_path = os.path.join(dirpath, "data_out")

filenames = os.listdir(data_path)
filenames.remove('missing_teats.csv')
filenames.remove('.ipynb_checkpoints')
mt_df = pd.read_csv(os.path.join(data_path, "missing_teats.csv"))
cow_list = np.unique(mt_df.cow)
len(cow_list)

38

In [43]:
for file in filenames:
    df = pd.read_csv(os.path.join(data_path, file)).set_index("cow")
    for cow in cow_list:
        teats_missing = mt_df[mt_df.cow == cow]["teat"].values
        for teat in teats_missing:
            columns_with_teat = [col for col in df.columns if teat in col]
            df.loc[cow, columns_with_teat] = np.nan
            # section = df.loc[cow, columns_with_teat]
    # save file 2.0
    df = df.reset_index()
    df.to_csv(os.path.join(data_path, "m" +file), index = False)

In [58]:
feature_path =  r"C:\Users\marie\rep_codes\udder_project\udder_processing\features_dict"
fdf = pd.read_csv(os.path.join(feature_path, "feature_table.csv"))
# cange values of missing teats to missing
for cow in cow_list:
    teats_missing = mt_df[mt_df.cow == cow]["teat"].values
    for teat in teats_missing:
        columns_with_teat = [col for col in fdf.columns if teat in col]
        fdf.loc[fdf.cow == cow, columns_with_teat] = np.nan
        temp = fdf[fdf.cow == cow][columns_with_teat]
        # print(temp)
    fdf.to_csv(os.path.join(feature_path, "mfeature_table.csv"))

C:\Users\marie\AppData\Local\Temp\ipykernel_21360\2951280191.py:2: DtypeWarning: Columns (21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  fdf = pd.read_csv(os.path.join(feature_path, "feature_table.csv"))


In [90]:
fdf[fdf.columns[3:]] = fdf[fdf.columns[3:]].apply(pd.to_numeric, errors='coerce')
fdfg = fdf.drop(["frame", "filename"], axis = 1).groupby(["cow"]).median()

fdfg_cols = fdfg.columns
for colname in fdfg_cols:
    # print(colname)
    Q1 = np.nanpercentile(fdfg[colname], 25, method='midpoint')
    Q3 = np.nanpercentile(fdfg[colname], 75, method='midpoint')
    med = np.nanpercentile(fdfg[colname], 50, method='midpoint')
    IQR = Q3 - Q1
    low_thres = med - 1.5*IQR
    up_thres = med + 1.5*IQR
    out = (fdfg[colname] < low_thres)| (fdfg[colname] > up_thres)
    fdfg.loc[out, colname] = np.nan

fdfg = fdfg.reset_index()
fdfg.to_csv(os.path.join(feature_path, "gmfeature_table.csv"))

udder_vol
lf_vol
rf_vol
lb_vol
rb_vol
udder_sarea
lf_sarea
rf_sarea
lb_sarea
rb_sarea
lf_angle
rf_angle
lb_angle
rb_angle
front_eu
back_eu
right_eu
left_eu
front_gd
back_gd
right_gd
left_gd
udder_peri
udder_area
udder_circ
udder_exc
lf_peri
lf_area
lf_circ
lf_exc
rf_peri
rf_area
rf_circ
rf_exc
lb_peri
lb_area
lb_circ
lb_exc
rb_peri
rb_area
rb_circ
rb_exc
lf_len
rf_len
lb_len
rb_len
